### Aggregation of federated VUS analyses.
This notebook aggregates the results from executing the VUS analysis on three datasets carried out remotely on different platforms.

See the accompanying notebooks in this folder for the individual analyses.

The following are the DRS ids for the analysis results in the three different systems where they were conducted.

In [1]:
resultsFiles = {'osteosarcoma':'sbcav:6188354bd2f88b031ee29d82',
               'copdgene': 'sbbdc:61888303e6261a31b6ddaf9b',
                'tcga':'sbcgc:6180182b9c5e581c1b22026b'
               }

In [2]:
from fasp.loc import DRSMetaResolver
drsClient = DRSMetaResolver()

Searching the GA4GH registry for org.ga4gh:drs services
GA4GH registry unavailable, cannot get registered DRS services.
Continuing with locally known DRS services.


In [3]:
import requests
import os
def download(url, source):
    response = requests.get(url)
    data = response.json()
    # flatten cooccurrence output
    flat_vus = []
    for k, v in data['cooccurring vus'].items():
        pathogenic_count = len(v['pathogenic variants'])
        ## this is a pythonic way of merging dicts - it is cryptic
        z = {**{"source":source},**{"vus":k}, **v['likelihood data'], **v['allele frequencies'], **{"no_pathogenic_coocurrs":pathogenic_count}}
        flat_vus.append(z)
    return flat_vus

In [4]:
flat_vus = []
for source, drsCurie in resultsFiles.items():
    url = drsClient.getAccessURL(drsCurie,'s3')
    flat_vus = flat_vus + download(url,source)

# turn the array of dicts into a data frame    
import pandas as pd
flat_df = pd.DataFrame(flat_vus)
flat_df

/Users/forei/imfenvs/fasp/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


,source,vus,p1,p2,n,k,likelihood,maxPop,maxPopFreq,cohortFreq,no_pathogenic_coocurrs
0,osteosarcoma,"('13', 32315831, 'G', 'A')",0.015152,0.001,28,1,0.097015,NaN,NaN,0.848485,1
1,osteosarcoma,"('13', 32318080, 'C', 'T')",0.015152,0.001,33,1,0.104188,NaN,NaN,1.000000,1
2,osteosarcoma,"('13', 32318598, 'T', 'C')",0.015152,0.001,32,1,0.102712,NaN,NaN,0.969697,1
3,osteosarcoma,"('13', 32321240, 'G', 'C')",0.015152,0.001,33,1,0.104188,NaN,NaN,1.000000,1
4,osteosarcoma,"('13', 32325741, 'C', 'T')",0.015152,0.001,33,1,0.104188,NaN,NaN,1.000000,1
...,...,...,...,...,...,...,...,...,...,...,...
108,tcga,"('13', 32968591, 'G', 'A')",0.000337,0.001,1383,1,1.186443,NaN,NaN,0.133122,1
109,tcga,"('13', 32962045, 'A', 'C')",0.000337,0.001,204,1,2.594216,NaN,NaN,0.019636,1
110,tcga,"('13', 32962088, 'C', 'T')",0.000337,0.001,144,1,2.699582,NaN,NaN,0.013861,1
111,tcga,"('13', 32893791, 'A', 'G')",0.000337,0.001,61,1,2.852431,NaN,NaN,0.005872,1


In [5]:
merged_df = flat_df.pivot(index='vus', columns='source', values=['n','cohortFreq','no_pathogenic_coocurrs'])

In [6]:
pd.set_option('display.max_rows',100)
merged_df

n                       cohortFreq  \
source                              copdgene osteosarcoma     tcga   copdgene   
vus                                                                             
('13', 32315831, 'G', 'A')            9345.0         28.0      NaN   0.914742   
('13', 32318080, 'C', 'T')           10195.0         33.0      NaN   0.997944   
('13', 32318598, 'T', 'C')            9047.0         32.0      NaN   0.885572   
('13', 32319654, 'A', 'G')            8119.0          NaN      NaN   0.794734   
('13', 32321240, 'G', 'C')           10196.0         33.0      NaN   0.998042   
('13', 32323151, 'ATT', 'A')          4106.0          NaN      NaN   0.401919   
('13', 32325741, 'C', 'T')           10194.0         33.0      NaN   0.997847   
('13', 32331128, 'G', 'A')           10194.0         33.0      NaN   0.997847   
('13', 32333969, 'A', 'G')           10194.0         33.0      NaN   0.997847   
('13', 32338918, 'A', 'G')           10194.0         33.0      NaN   0.997847   
('13', 32340868, 'G', 'C')           10193.0         33.0      NaN   0.997749   
('13', 32342270, 'CAAA', 'CA')           NaN         24.0      NaN        NaN   
('13', 32343709, 'G', 'GA')            635.0          NaN      NaN   0.062157   
('13', 32343709, 'GA', 'G')           3580.0          NaN      NaN   0.350431   
('13', 32344166, 'GA', 'G')           7460.0          NaN      NaN   0.730227   
('13', 32345879, 'G', 'A')           10194.0         33.0      NaN   0.997847   
('13', 32346707, 'T', 'C')           10194.0         33.0      NaN   0.997847   
('13', 32353757, 'C', 'T')            5163.0          NaN      NaN   0.505384   
('13', 32354180, 'T', 'C')           10202.0         33.0      NaN   0.998630   
('13', 32354271, 'G', 'A')            8080.0          NaN      NaN   0.790916   
('13', 32355250, 'T', 'C')           10202.0         33.0      NaN   0.998630   
('13', 32356799, 'G', 'A')           10216.0         33.0      NaN   1.000000   
('13', 32359222, 'GAAA', 'GA')           NaN         10.0      NaN        NaN   
('13', 32359800, 'A', 'G')           10202.0         33.0      NaN   0.998630   
('13', 32362509, 'T', 'C')            8062.0          NaN      NaN   0.789154   
('13', 32365177, 'C', 'T')            7875.0          NaN      NaN   0.770850   
('13', 32366208, 'C', 'A')            7873.0         25.0      NaN   0.770654   
('13', 32367579, 'A', 'AGATGGCTTG')   7883.0          NaN      NaN   0.771633   
('13', 32371874, 'A', 'G')           10202.0         32.0      NaN   0.998630   
('13', 32372278, 'C', 'T')           10202.0         33.0      NaN   0.998630   
('13', 32375245, 'T', 'C')            8016.0          NaN      NaN   0.784652   
('13', 32375563, 'C', 'CT')           7971.0          NaN      NaN   0.780247   
('13', 32376120, 'A', 'G')            7787.0          NaN      NaN   0.762236   
('13', 32377590, 'CAAA', 'CAA')          NaN         30.0      NaN        NaN   
('13', 32379251, 'T', 'C')            7834.0          NaN      NaN   0.766836   
('13', 32380424, 'G', 'A')            7992.0          NaN      NaN   0.782302   
('13', 32380604, 'T', 'A')           10202.0         33.0      NaN   0.998630   
('13', 32381260, 'C', 'T')            7827.0          NaN      NaN   0.766151   
('13', 32383835, 'G', 'C')               NaN         13.0      NaN        NaN   
('13', 32385062, 'C', 'T')            7826.0          NaN      NaN   0.766053   
('13', 32385702, 'A', 'C')            7830.0          NaN      NaN   0.766445   
('13', 32386226, 'T', 'G')            8009.0          NaN      NaN   0.783966   
('13', 32387052, 'G', 'GC')          10202.0         33.0      NaN   0.998630   
('13', 32387908, 'A', 'C')            7816.0          NaN      NaN   0.765074   
('13', 32387951, 'C', 'T')            7826.0          NaN      NaN   0.766053   
('13', 32388121, 'C', 'CT')           9942.0          NaN      NaN   0.973179   
('13', 32388121, 'CTT', 'CT')            NaN         14.0      NaN        

Note, the TCGA locations are in GRCh37 coordinates. The cooccurrence code uses the location as the identifier of the vus. Need to match variants across reference genomes. 